In [1]:
import sys
import flap

sys.path.append(r"E:\marci main\Fusion\nti-wavelet-tools\python\utility")
import convert_dict_to_flap
sys.path.append(r"E:\marci main\Fusion\nti-wavelet-tools\python\core")
import core

INIT flap storage


In [7]:
path = r"E:\marci main\fusion\inputdata\AUGD_38088_Loaded-with-MTR_processed_CWT_smoothed.sav"
print(path)

data = core.NWTDataObject()
data.load_proc_sav(path)

E:\marci main\fusion\inputdata\AUGD_38088_Loaded-with-MTR_processed_CWT_smoothed.sav


In [8]:
print(data.raw_data.data)
flap.list_data_objects(data.raw_data)

[[ 16.456413    17.769232    18.958973   ...   0.83591723   1.1333586
    1.0513043 ]
 [ -4.7538133   -5.071766    -5.1845875  ...  -0.2512583   -0.43587017
   -0.63074577]
 [ -8.671757    -8.682013    -8.784578   ...  -0.11792531  -0.21023524
   -0.23074678]
 ...
 [-11.974322   -11.974322   -11.451246   ...  -0.2615141   -0.31279293
   -0.2615141 ]
 [ -3.3339987  -15.077357    -9.139336   ...  -5.954685    -6.18691
   -5.921514  ]
 [ 11.748728    13.912824    15.738462   ...   0.6205381    0.52822816
    0.5487397 ]]

-----------------------------
<1>(data_source:"" exp_id:"AUGD-38088") data_title:"raw data" shape:[8,8001][no error]
  Data name:"unit", unit:"a.u."
  Coords:
    'Time [s](Dims:1, Shape:[8001]) [<R. symm.>] Val. range:  1.004E+00 -  1.008E+00
    'Channels [a.u.](Dims:0, Shape:[8]) [<R. symm.>] Val:MHI-B31-40, MHI-B31-14, MHI-B31-03, MHI-B31-01, MHI-B31-02, MHI-B31-12, MHI-B31-22, MHI-B31-13
    'Theta [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 6.163E+00,  6.206E+00,  6

'\n-----------------------------\n<1>(data_source:"" exp_id:"AUGD-38088") data_title:"raw data" shape:[8,8001][no error]\n  Data name:"unit", unit:"a.u."\n  Coords:\n    \'Time [s](Dims:1, Shape:[8001]) [<R. symm.>] Val. range:  1.004E+00 -  1.008E+00\n    \'Channels [a.u.](Dims:0, Shape:[8]) [<R. symm.>] Val:MHI-B31-40, MHI-B31-14, MHI-B31-03, MHI-B31-01, MHI-B31-02, MHI-B31-12, MHI-B31-22, MHI-B31-13\n    \'Theta [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 6.163E+00,  6.206E+00,  6.201E+00,  6.206E+00,  6.217E+00,  6.204E+00,  6.159E+00,  6.206E+00\n    \'Phi [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 2.791E+00,  3.596E+00,  4.743E+00,  4.773E+00,  5.184E+00,  5.747E+00,  5.969E+00,  6.753E+00'

In [9]:
path2 = r"E:\marci main\fusion\inputdata\proba.pynwt"
data.save(path2)

In [10]:
data2 = core.NWTDataObject()
data2.load(path2)
data2.save(r"E:\marci main\fusion\inputdata\proba2.pynwt")
flap.list_data_objects(data2.raw_data)


-----------------------------
<1>(data_source:"" exp_id:"AUGD-38088") data_title:"raw data" shape:[8,8001][no error]
  Data name:"unit", unit:"a.u."
  Coords:
    'Time [s](Dims:1, Shape:[8001]) [<R. symm.>] Val. range:  1.004E+00 -  1.008E+00
    'Channels [a.u.](Dims:0, Shape:[8]) [<R. symm.>] Val:MHI-B31-40, MHI-B31-14, MHI-B31-03, MHI-B31-01, MHI-B31-02, MHI-B31-12, MHI-B31-22, MHI-B31-13
    'Theta [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 6.163E+00,  6.206E+00,  6.201E+00,  6.206E+00,  6.217E+00,  6.204E+00,  6.159E+00,  6.206E+00
    'Phi [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 2.791E+00,  3.596E+00,  4.743E+00,  4.773E+00,  5.184E+00,  5.747E+00,  5.969E+00,  6.753E+00


'\n-----------------------------\n<1>(data_source:"" exp_id:"AUGD-38088") data_title:"raw data" shape:[8,8001][no error]\n  Data name:"unit", unit:"a.u."\n  Coords:\n    \'Time [s](Dims:1, Shape:[8001]) [<R. symm.>] Val. range:  1.004E+00 -  1.008E+00\n    \'Channels [a.u.](Dims:0, Shape:[8]) [<R. symm.>] Val:MHI-B31-40, MHI-B31-14, MHI-B31-03, MHI-B31-01, MHI-B31-02, MHI-B31-12, MHI-B31-22, MHI-B31-13\n    \'Theta [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 6.163E+00,  6.206E+00,  6.201E+00,  6.206E+00,  6.217E+00,  6.204E+00,  6.159E+00,  6.206E+00\n    \'Phi [rad](Dims:0, Shape:[8]) [<R. symm.>] Val: 2.791E+00,  3.596E+00,  4.743E+00,  4.773E+00,  5.184E+00,  5.747E+00,  5.969E+00,  6.753E+00'